# ML time series missing data imputation

This Jupyter Notebook contains multiple data imputation techniques that are natively implemented by sklearn and pandas.

## Importing useful modules and packages

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
%matplotlib inline
from time import time

pd.set_option('display.max_columns', None)

## Importing the dataset

Importation of the dataset, renaming columns and sorting the values by date.

In [ ]:
original_data = pd.read_csv("PJME_hourly.csv")
original_data["Datetime"] = pd.to_datetime(original_data["Datetime"])
original_data.rename({"Datetime" : "date", "PJME_MW" : "out"}, axis=1, inplace=True)
original_data.sort_values("date", ascending=True, inplace=True, ignore_index=True)
original_data.set_index("date", inplace=True)

Removing dupplicated row (we keep the first one)

In [ ]:
pd.concat([original_data[original_data.index.duplicated(keep="first") == True],
          original_data[original_data.index.duplicated(keep="last") == True]])

In [ ]:
original_data = original_data[~original_data.index.duplicated(keep='first')]

## Checking for missing dates

Checking for missing dates in the data set (date than have never been written, to add them with the value na). We want to have all the dates to have a frequency of 1H and being able to use imputation method of sklearn.

In [ ]:
data_gap = original_data.copy()

startDate = original_data.iloc[0].name
endDate = original_data.iloc[-1].name

dateRange = pd.date_range(startDate, endDate, freq='1h')
h = pd.Timedelta("1h")

df_missing = pd.DataFrame(dateRange.difference(data_gap.index), columns=["date"]).set_index("date")
df_missing['out'] = np.nan
data_gap = pd.concat([data_gap, df_missing])
data_gap['index'] = data_gap.reset_index().index
data_gap.isna().sum()

Making sure the data is sorted by date and with a frequency of 1h

In [ ]:
data_gap.sort_index(inplace=True)
data_gap = data_gap.asfreq('1h')

## Creating missing values

We want to have missing values with large time window because there is large time window with missing data in the datasets provided by the the different sources of the project.

In [ ]:
row_number = len(data_gap)

def index_to_date(index_list, start=startDate, end=endDate):
    index_list = list(index_list)
    for i in range(len(index_list)):
        index_list[i] = startDate + index_list[i] * h
    return index_list

def date_to_index(date):
    return (date - startDate) / h

start_missing_date = pd.to_datetime("2007-05-01 01:00:00")
end_missing_date = pd.to_datetime("2007-05-07 23:00:00")
missing_data_interval = pd.date_range(start_missing_date, end_missing_date, freq='1h')
N_missing_data = int(0.5 * ((end_missing_date - start_missing_date)/h))

missing_index = index_to_date(np.random.randint(date_to_index(start_missing_date), date_to_index(end_missing_date), size=N_missing_data))
data_gap.loc[:, 'gap'] = data_gap['out'].copy()
data_gap.loc[missing_index, 'gap'] = np.nan

In [ ]:
row_number = len(data_gap)

def index_to_date(index, start=startDate, end=endDate):
    return startDate + index * h

def date_to_index(date):
    return (date - startDate) / h

start_missing_date = pd.to_datetime("2007-05-01 01:00:00")
end_missing_date = pd.to_datetime("2007-05-07 23:00:00")
seg_len = 2
nb_gap = 25
missing_data_interval = pd.date_range(start_missing_date, end_missing_date, freq='1h')
start_index = date_to_index(start_missing_date)
end_index = date_to_index(end_missing_date)

data_gap.loc[:, 'gap'] = data_gap['out'].copy()

for _ in range(nb_gap):
    seg_start = np.random.randint(start_index, end_index)
    seg_stop = seg_start + seg_len
    
    seg_start_date = index_to_date(seg_start)
    seg_stop_date = index_to_date(seg_stop)
    
    data_gap.loc[seg_start_date:seg_stop_date, 'gap'] = np.nan

Visualisation of the missing data over one week

In [ ]:
plt.title("Creating missing values")
data_gap['2007 05 03':'2007 05 07']['gap'].plot(style="r", alpha=1, linewidth=4)
data_gap['2007 05 03':'2007 05 07']['out'].plot(style="b", alpha=0.6)
plt.legend(["Data with missing values", 'Real'])
plt.xlabel("Date")

plt.show()

## Creating date features

Creating features that are usefull for some interpolation techniques.

In [ ]:
def get_features(df):
    out = df.copy()
    out["hour"] = out.index.hour
    out["day"] = out.index.day
    out["month"] = out.index.month
    out["year"] = out.index.year
    
    out['quarter'] = out.index.quarter
    out['dayofyear'] = out.index.dayofyear
    out['dayofmonth'] = out.index.day
    
    out['weekofyear'] = out.index.isocalendar().week.astype(np.int64)
    return out

data_gap = get_features(data_gap)
data_gap

## Imputation

Importing the sklearn imputer and initializing those that are experimentals. And creating the targets/features used for the imputation. Those are the features that will be used to fill the missing data and also the features where the missing data will be checked to be imputed.

In [ ]:
from sklearn.experimental import enable_iterative_imputer # The implementation of some imputer are experimentals
from sklearn.impute import IterativeImputer, KNNImputer, \
                            MissingIndicator, SimpleImputer
target = ['gap', 'index', 'hour', 'day', 'month', 'year', 'quarter', 'dayofyear', 'dayofmonth', 'weekofyear']
target = ['gap', 'index']

data_gap['index'] = data_gap.reset_index().index

In [ ]:
period1 = '2007 05 03'
period2 = '2007 05 07'

### Simple Imputer

The single imputer is computing the mean and using it to fill the missing values, as you can guess it is bad for time series forecasting.

In [ ]:
simple_imputer = SimpleImputer()
simple_imputer.set_output(transform="pandas")
data_gap['simple'] = simple_imputer.fit_transform(data_gap.loc[period1:period2, target])['gap']

In [ ]:
plt.title("Gap filled with simple imputer")
data_gap[period1:period2]['out'].plot(style="b", alpha=1, linewidth=4)
data_gap[period1:period2]['simple'].plot(style="r", alpha=0.6)
plt.legend(["Real", 'Missing values filled'])
plt.xlabel("Date")

plt.show()

### Iterative imputer

The single imputer is computing the mean of the similar row (row with similar features values) and using it to fill the missing values, as you can guess it is bad for time series forecasting. This mean the result may vary depending on the choosen target.

In [ ]:
ite_imputer = IterativeImputer(initial_strategy='median')
ite_imputer.set_output(transform="pandas")
data_gap['Iterative'] = ite_imputer.fit_transform(data_gap.loc[period1:period2, target])['gap']

In [ ]:
plt.title("Gap filled with iterative imputer")
data_gap[period1:period2]['out'].plot(style="b", alpha=1, linewidth=4)
data_gap[period1:period2]['Iterative'].plot(style="r", alpha=0.6)
plt.legend(["Real", 'Missing values filled'])
plt.xlabel("Date")

plt.show()

### KNNImputer

The KNNImputer known as K-Nearest Neighbors imputer is using the features to define which rows are close on fill the missing value using the K closest values. It is possible to use KNNImputer with customs weights and metric to improve the results to our dataset. However, only the native metrics and weights are used below.

In [ ]:
n_neighbors = 10

In [ ]:
knn_imputer_unif = KNNImputer(n_neighbors=n_neighbors, weights="uniform")
knn_imputer_unif.set_output(transform="pandas")
data_gap['KNN_unif'] = knn_imputer_unif.fit_transform(data_gap.loc[period1:period2, target])['gap']

In [ ]:
knn_imputer_dist = KNNImputer(n_neighbors=n_neighbors, weights="distance")
knn_imputer_dist.set_output(transform="pandas")
data_gap['KNN_dist'] = knn_imputer_dist.fit_transform(data_gap.loc[period1:period2, target])['gap']

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(15, 5*4), sharex=True)

data_gap[period1:period2]['out'].plot(ax=ax[0], style="b", alpha=1, linewidth=3)
data_gap[period1:period2]['KNN_unif'].plot(ax=ax[0], style="r", alpha=0.6)
data_gap[period1:period2]['KNN_dist'].plot(ax=ax[0], style="g", alpha=0.6)
ax[0].legend(["Real", 'Uniform', 'Distance', 'Custom'])
ax[0].set_title("Filling missing values using uniform and distance distribution")

data_gap[period1:period2]['out'].plot(ax=ax[1], style="b", alpha=1, linewidth=3)
data_gap[period1:period2]['KNN_dist'].plot(ax=ax[1], style="r", alpha=0.6)
ax[1].legend(["Real", 'Missing values filled'])
ax[1].set_title("Filling missing values using a distance distribution")

data_gap[period1:period2]['out'].plot(ax=ax[2], style="b", alpha=1, linewidth=3)
data_gap[period1:period2]['KNN_unif'].plot(ax=ax[2], style="r", alpha=0.6)
ax[2].legend(["Real", 'Missing values filled'])
ax[2].set_title("Filling missing values using uniform distribution")

plt.legend(["Real", 'Missing values filled'])
plt.title("Gap filled with KNN imputer")
plt.show()

### Pandas imputation

#### Forward fill (ffill)

The forward fill method of pandas allow the user to "propagate last valid observation forward to next valid". This is a bad method for time series when the length of the missing values is long.

In [ ]:
data_gap['pandas_ffill'] = data_gap['gap'].copy()
data_gap['pandas_ffill'] = data_gap['pandas_ffill'].fillna(method='ffill')

In [ ]:
data_gap[period1:period2]['out'].plot(style="b", alpha=1, linewidth=4)
data_gap[period1:period2]['pandas_ffill'].plot(style="r", alpha=0.6)
plt.legend(["Real", 'Missing values filled'])
plt.title("Filling missing values using forward filling")
plt.xlabel("Date")

plt.show()

#### Backward fill (bfill)

The backward fill method of pandas allow the user to "use next valid observation to fill gap". This is a bad method for time series when the length of the missing values is long.

In [ ]:
data_gap['pandas_bfill'] = data_gap['gap'].copy()
data_gap['pandas_bfill'] = data_gap['pandas_bfill'].fillna(method='bfill')

In [ ]:
data_gap[period1:period2]['out'].plot(style="b", alpha=1, linewidth=4)
data_gap[period1:period2]['pandas_bfill'].plot(style="r", alpha=0.6)
plt.legend(["Real", 'Missing values filled'])
plt.xlabel("Date")
plt.title("Filling missing values using backward filling")

plt.show()

### Pandas interpolate

In this section the following interpolating method of pandas are used :

- linear
- polynomial

#### Linear

A linear regression is a way of imputing missing data by drawing a line between 2 known points and affect the values of the line to the missing points.

In [ ]:
data_gap['pandas_linear'] = data_gap['gap'].copy()
data_gap['pandas_linear'] = data_gap['pandas_linear'][period1:period2].interpolate(method='linear')

In [ ]:
data_gap[period1:period2]['out'].plot(style="b", alpha=1, linewidth=4)
data_gap[period1:period2]['pandas_linear'].plot(style="r", alpha=0.6)
plt.legend(["Data", 'Linear interpolation values'])
plt.title("Linear interpolation")

plt.show()

#### Polynomial

A polynomial interpolation is a way of fitting a polynom of a given order to the known data to then impute the missing ones.

In [ ]:
orders = range(3, 8, 2)

plt.plot(data_gap[period1:period2]['out'], color="b", alpha=1, linewidth=4)

legends = ["Real"]

for order in orders:
    data_gap['pandas_polynomial_' + str(order)] = data_gap['gap'].copy()
    data_gap['pandas_polynomial_' + str(order)] = data_gap['pandas_polynomial_' + str(order)][period1:period2].interpolate(method='polynomial', order=order)


    plt.plot(data_gap[period1:period2]['pandas_polynomial_' + str(order)], alpha=1, linewidth=1)
    legends.append("Order " + str(order))

plt.title("Gap filled with pandas polynomial interpolation")
plt.legend(["Real"] + list(orders))
plt.xlabel("Date")

plt.show()

In [ ]:
orders = range(3, 16, 2)

fig, ax = plt.subplots(len(orders) + 1, 1, figsize=(15, 5*len(orders)), sharex=True)
data_gap[period1:period2]['out'].plot(ax=ax[0], style="b", alpha=1, linewidth=3)

i = 1
for order in orders:
    data_gap['pandas_polynomial_' + str(order)] = data_gap['gap'].copy()
    data_gap['pandas_polynomial_' + str(order)] = data_gap['pandas_polynomial_' + str(order)].interpolate(method='polynomial', order=order)
    
    # Print all
    data_gap[period1:period2]['pandas_polynomial_' + str(order)].plot(ax=ax[0], alpha=1, linewidth=1)
    
    # Print only new graph
    data_gap[period1:period2]['out'].plot(ax=ax[i], style="b", alpha=1, linewidth=3)
    data_gap[period1:period2]['pandas_polynomial_' + str(order)].plot(ax=ax[i], style="r", alpha=0.6)
    ax[i].legend(["Real", 'Missing values filled'])
    ax[i].set_title("Gap filled with pandas polynomial interpolation and order = " + str(order))
    i += 1
ax[0].legend(["Real"] + list(orders))

plt.tight_layout()
plt.show()